# 01 - EEG Temporal Decoding

Time-resolved classification of reward vs. no-reward feedback using REWP.

**Based on**: B_TMAZE_EEG_STANDARD_CLASS notebooks

**Key Analysis:**
1. Load preprocessed EEG epochs
2. Time-resolved decoding with SlidingEstimator
3. Identify significant time windows
4. Focus on REWP window (240-340ms)

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

# T-Maze analysis modules
from core.containers import TMazeEEGData
from io.loaders import TMazeEEGLoader
from classification.temporal import (
    temporal_decoding,
    find_significant_times,
    rewp_temporal_analysis
)
from visualization.plots import plot_temporal_decoding, plot_rewp_waveform

## 1. Load EEG Data

Load preprocessed MNE epochs with reward/no-reward labels.

In [ ]:
# Example with your data paths
# Adjust paths to your actual epoch files

# Define condition mapping (Reward=1, NoReward=0)
condition_mapping = {
    'MazeReward': 1,
    'NoMazeReward': 1,
    'MazeNoReward': 0,
    'NoMazeNoReward': 0
}

# Create loader
loader = TMazeEEGLoader(
    condition_mapping=condition_mapping,
    tmin=-0.2,
    tmax=0.8,
    baseline=(-0.2, 0)
)

# Load epochs (uncomment with your data)
# eeg_data = loader.load("/path/to/sub-01-epo.fif", subject_id="sub-01")

print("Loader configured. Ready to load epochs.")

## 2. Create Simulated Data (Demo)

For demonstration, create synthetic EEG data with reward-related signal.

In [ ]:
# Simulate EEG data for demo
np.random.seed(42)

n_epochs = 200
n_channels = 64
sfreq = 200
tmin, tmax = -0.2, 0.8
n_times = int((tmax - tmin) * sfreq)
times = np.linspace(tmin, tmax, n_times)

# Generate random data
X = np.random.randn(n_epochs, n_channels, n_times) * 5
y = np.random.randint(0, 2, n_epochs)

# Add reward signal in REWP window (240-340ms) for reward trials
rewp_mask = (times >= 0.24) & (times <= 0.34)
for i in range(n_epochs):
    if y[i] == 1:  # Reward
        X[i, :10, rewp_mask] += 3  # Add signal to frontal channels

# Create data container
channels = [f'Ch{i}' for i in range(n_channels)]
channels[:5] = ['FCz', 'Fz', 'Cz', 'FC1', 'FC2']  # Name FCz-region channels

eeg_data = TMazeEEGData(
    data=X,
    times=times,
    labels=y,
    condition_names=['NoReward', 'Reward'],
    channels=channels,
    sfreq=sfreq,
    subject_id='demo'
)

print(eeg_data)

## 3. Time-Resolved Decoding

In [ ]:
# Run temporal decoding
    eeg_data,
    classifier_type='svm',
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=True
)

print(f"\nPeak decoding: {result.peak_score:.3f} at {result.peak_time*1000:.0f}ms")

In [ ]:
# Plot temporal decoding results
plot_temporal_decoding(
    result,
    chance_level=0.5,
    title='Reward vs. No-Reward Temporal Decoding'
)
plt.show()

## 4. Find Significant Time Windows

In [ ]:
# Find significant time periods
sig_mask, clusters = find_significant_times(
    result,
    chance_level=0.5,
    alpha=0.05,
    method='ttest'
)

print("Significant time clusters:")
for start, end in clusters:
    print(f"  {start*1000:.0f}ms to {end*1000:.0f}ms")

if result.onset_time:
    print(f"\nOnset of significant decoding: {result.onset_time*1000:.0f}ms")

In [ ]:
# Plot with significance highlighted
plot_temporal_decoding(
    result,
    chance_level=0.5,
    show_significance=True,
    title='Temporal Decoding with Significant Periods'
)
plt.show()

## 5. REWP-Focused Analysis

Focus on frontocentral channels and REWP time window.

In [ ]:
# REWP-specific analysis
rewp_result = rewp_temporal_analysis(
    eeg_data,
    fcz_only=True,
    classifier_type='svm',
    cv=5
)

print(f"REWP Analysis (FCz channels only):")
print(f"  Peak: {rewp_result.peak_score:.3f} at {rewp_result.peak_time*1000:.0f}ms")

# Find significant periods in REWP window
find_significant_times(rewp_result, chance_level=0.5)

In [ ]:
# Plot REWP decoding
plot_temporal_decoding(
    rewp_result,
    title='REWP Decoding (FCz Channels, 0-500ms)'
)
plt.show()

## 6. ERP Visualization

In [ ]:
# Plot grand average waveforms
plot_rewp_waveform(
    eeg_data,
    channels=['FCz', 'Fz', 'Cz'],
    title='Grand Average REWP Waveform'
)
plt.show()

## 7. Multi-Subject Analysis Loop

Template for running across subjects.

In [ ]:
# Example multi-subject analysis
# Uncomment and modify with your actual paths

# subject_paths = [
#     "/path/to/sub-01-epo.fif",
#     "/path/to/sub-02-epo.fif",
#     # ...
# ]

# all_results = []
# for path in subject_paths:
#     subj_id = path.split('/')[-1].split('-')[0]
#     eeg_data = loader.load(path, subject_id=subj_id)
#     result = temporal_decoding(eeg_data, classifier_type='svm')
#     all_results.append(result)
#     print(f"{subj_id}: Peak={result.peak_score:.2f} at {result.peak_time*1000:.0f}ms")

print("Multi-subject loop template ready.")

## Summary

This notebook demonstrated:
1. Loading EEG epochs with reward condition labels
2. Time-resolved decoding using SlidingEstimator
3. Identifying significant decoding periods
4. REWP-focused analysis (FCz channels, 240-340ms)

### Key Findings Template:
- Peak decoding at ~XXX ms (within REWP window)
- Significant decoding from XXX to XXX ms
- Decoding AUC: 0.XX

### Next Steps:
- **02_fmri_roi_classification.ipynb**: fMRI ROI analysis